# Introduction: Thomas Fire. Began December 4, 2017. Fueled by Santa Ana winds.
The Thomas Fire began December 4, 2017 near the town of Santa Paula (34.4267,-119.1592). It grew into what, at the time, was California's largest fire in modern history. Let's use the API to explore the weather conditions in the days around the start of the fire that contributed to it's growth. 

## Q1: What were the winds like in the days leading up to the fire? How sustained were they?

## Q2: How did the wind, relative humidity, and temperature interact in ways that might have affected fire weather conditions?

## Q3: Were these conditions unprecedented over the last month?


# Step 0: Importing and function definitions

In [ ]:
import urllib.request as req
import json
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

We've built a couple of functions to help in building the API request and parsing the returned data. These use the Pandas python package and builds a DataFrame using with two indices: station id and dattim

In [ ]:
def return_station_df(data, service):
    """Build pandas dataframes for data and metadata using json response from 
        requests to Time Series, Nearest, and Latest services

    Parameters:
        data: dict, json response from API request
        date_format: str, requested date format
        service: str, Synoptic web service requested
    
    Returns:
        data_df: pandas DataFrame, data return from all station
        meta_df: pandas DataFrame, station metadata
    """
    dattim_format = '%Y-%m-%d %H:%M'
    meta_list = []
    for i in range(len(data)):
        # Metadata
        stid = data[i]['STID']
        mnet_id = data[i]['MNET_ID']
        try:
            lon = float(data[i]['LONGITUDE'])
        except TypeError:
            lon = None
        try:
            lat = float(data[i]['LATITUDE'])
        except TypeError:
            lat = None
        try:
            elev = float(data[i]['ELEVATION'])
        except TypeError:
            elev = None
        meta_list.append([stid, mnet_id, lon, lat, elev])

        # Data
        data_out = data[i]['OBSERVATIONS'].copy()
        if service == 'timeseries':
            datetime = pd.to_datetime(data_out['date_time'], format=(dattim_format))
            del data_out['date_time']
            multi_index = pd.MultiIndex.from_product([[stid], datetime],
                                                     names=["stid", "dattim"])
        else:
            datetime = pd.to_datetime(data_out[list(data_out.keys())[0]]['date_time'], format=(dattim_format))
            for key in data_out:
                data_out.update({key: data_out[key]['value']})
            multi_index = pd.MultiIndex.from_arrays([[stid], [datetime]],
                                                    names=["stid", "dattim"])

        #Concatenate as needed
        if i == 0:
            data_df = pd.DataFrame(data_out, index=multi_index)
        else:
            data_df = pd.concat([data_df, pd.DataFrame(data_out, index=multi_index)], axis=0)

    #Build metadata dataframe from list
    meta_df = pd.DataFrame(meta_list, columns=["stid", "mnet_id", "lon", "lat", "elev"])
    meta_df.set_index('stid', inplace=True)

    # Sort the resulting data dataframe by time
    data_df.sort_index(inplace=True)

    return data_df, meta_df

In [ ]:
def make_api_request(url, api_args):
    for argument, value in api_args.items():
        url = url + '&' + argument + '=' + value

    with req.urlopen(url) as response:
        body = response.read()

    try:
        output = json.loads(body)
    except:
        decoded_body = body.decode('latin1')
        output = json.loads(decoded_body)

    return output

# Step 1: Build and make the API request

Let's build the api request we want to make

In [ ]:
token = ''
url = 'https://api.synopticdata.com/v2/stations/timeseries?'
api_args = {'radius': '34.427,-119.159,20',
            'start': '201712030000',
            'end': '201712050000',
            'obtimezone': 'local',
            'token': token,
            'vars': 'air_temp,wind_gust,wind_speed,relative_humidity'}

Make the request

In [ ]:
data = make_api_request(url, api_args)

# Step 2: Some initial insights into the data return

Let's start by confirming the contents have the keys we reviewed in the Explore exercise

In [ ]:
data.keys()


dict_keys(['UNITS', 'QC_SUMMARY', 'STATION', 'SUMMARY'])

... we can quickly view the number of objects (stations) have been returned

In [ ]:
# TODO Print the summary

... and do things like get a glimpse of the units returned for our requested variables

In [ ]:
# TODO Print the units

Now let's build out the DataFrames for the metadata and data return

In [ ]:
data_df, meta_df = return_station_df(data['STATION'], 'timeseries')


In [ ]:
meta_df

How many unique networks do these stations encompass?

In [ ]:
meta_df['mnet_id'].unique()

Alright! Let's take a quick look at the data... timeseries data for each particular station. 

In [ ]:
data_df

# Q1a: what were peak wind gusts like in the days around the start of the fire?
Let's start by checking the max wind gust across stations

In [ ]:
max_wind_gust = data_df['wind_gust_set_1'].groupby(level=0).max()
max_wind_gust.sort_values(ascending=False)

Woah. Some gusts up to 60 mph! 

Out of curiousity, is there any relationship between the wind gust and station elevation? Did these winds originate at high elevations across the area or might they have been more localized in nature?

In [ ]:
wind_elev = pd.concat([max_wind_gust, meta_df['elev']], axis=1)
fig0 = plt.figure(0, figsize=(12,6))
ax0 = fig0.add_subplot(111)
ax0.plot(wind_elev['elev'], wind_elev['wind_gust_set_1'], 'ko')
ax0.set_xlabel(f"Station Elevation ({data['UNITS']['elevation']})")
ax0.set_ylabel(f"Max wind gust ({data['UNITS']['wind_gust']})")

# Q1b: How sustained were winds?

Let's take a closer look at the winds of these windiest stations to get better situational awareness of when these started to ramp up.


In [ ]:
fig1 = plt.figure(1, figsize=(12,6))
ax = fig1.add_subplot(111)
ax.plot(data_df.loc['WLYC1'].index, data_df.loc['WLYC1','wind_speed_set_1'], 'r-', label='WLYC1')
ax.plot(data_df.loc['AT184'].index, data_df.loc['AT184','wind_speed_set_1'], 'k-', label='AT184')
ax.plot(data_df.loc['KCMA'].index, data_df.loc['KCMA','wind_speed_set_1'], 'b-', label='KCMA')
ax.legend()
ax.set_xlabel('Local Date/time')
ax.set_ylabel(f"Wind speed ({data['UNITS']['wind_speed']})")


# Question 2: How might the interaction of wind, RH, and temperature have affected fire conditions?

In [ ]:
fig2 = plt.figure(2, figsize=(12,6))
ax2 = fig2.add_subplot(111)
ax2b = ax2.twinx()
ax2c = ax2.twinx()

#Offset the 3rd axis
ax2c.spines['right'].set_position(("axes", 1.1))

# Plot
ax2.plot(data_df.loc['WLYC1'].index, data_df.loc['WLYC1','wind_speed_set_1'], 'b-', label='wind_speed')
ax2b.plot(data_df.loc['WLYC1'].index, data_df.loc['WLYC1','relative_humidity_set_1'], 'r-', label='RH')
ax2c.plot(data_df.loc['WLYC1'].index, data_df.loc['WLYC1','air_temp_set_1'], 'g-', label='Air temp')

ax2.set_ylim(-20,20)
ax2b.set_ylim(0,100)

# Set axis label colors
ax2.tick_params(axis='y', colors='b')
ax2b.tick_params(axis='y', colors='r')
ax2c.tick_params(axis='y', colors='g')

ax2.set_ylabel(f"wind speed ({data['UNITS']['wind_speed']})", color='b')
ax2b.set_ylabel(f"RH ({data['UNITS']['relative_humidity']})", color='r')
ax2c.set_ylabel(f"air temp ({data['UNITS']['air_temp']})", color='g')


# Question 3: Were these wind & RH values anomalous with respect to the past month?

Let's focus in a little more on a station and explore wind and RH over the month leading up to the Thomas Fire. 